# Lecture 10:
<div style="border: 1px double black; padding: 10px; margin: 10px">

**Goals for today's lecture:**
* More on joins
* Learn SQL (Structured Query Language)
</div>


In [1]:
install.packages('RSQLite')  # if required
install.packages('nycflights13')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(DBI)
library(dbplyr)
library(tidyverse)
library(nycflights13)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::ident()  masks dbplyr::ident()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::sql()    masks dbplyr::sql()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### Exercise
Assume every flight is full. The busiest travel day in terms of total # of passengers scheduled to depart is:
(Hint: the `planes` table tells you how many passengers each airplane holds.)

In [9]:
# code here
flights %>%
  left_join(planes, join_by(tailnum)) %>%
    group_by(month, day) %>%
      summarise(total = sum(seats, na.rm = T)) %>%
        arrange(-total)

`summarise()` has grouped output by 'month'. You can override using the
`.groups` argument.


month,day,total
<int>,<int>,<int>
11,27,118869
12,2,117736
11,26,117470
10,18,117324
10,21,117166
7,26,116944
11,7,116734
4,3,116716
11,21,116577


### Anti-join
`anti_join(x, y)` does the opposite of `semi_join`: it *drops* all the observations in `x` that are also in `y`.
![anti join](https://r4ds.hadley.nz/diagrams/join/anti.png)

Anti-joins are useful for finding mismatches in joins. There are many flights for which there is no tailnum in 'planes' table. To find out more about these flights we could use an anti-join to select only those rows:

In [13]:
anti_join(
    flights,
    planes,
    join_by("tailnum")
) %>% filter(!is.na(tailnum)) %>%
    distinct(tailnum) %>%
    print

# A tibble: 721 × 1
   tailnum
   <chr>  
 1 N3ALAA 
 2 N3DUAA 
 3 N542MQ 
 4 N730MQ 
 5 N9EAMQ 
 6 N532UA 
 7 N3EMAA 
 8 N518MQ 
 9 N3BAAA 
10 N3CYAA 
# ℹ 711 more rows



What could be the reason for missing tailnums in the planes dataframe?

In [14]:
flights %>% filter(tailnum == "N3ALAA") %>% print(width = Inf)

# A tibble: 63 × 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      558            600        -2      753            745
 2  2013     1     2     1907           1820        47     2037           2005
 3  2013     1     3      600            600         0      736            745
 4  2013     1     7     1911           1920        -9     2042           2100
 5  2013     1     8     1718           1720        -2     1900           1915
 6  2013     1    16      643            600        43      829            745
 7  2013     1    20     1829           1820         9     2007           2005
 8  2013     1    22     1713           1720        -7     1915           1915
 9  2013    10    11      624            629        -5      907            929
10  2013    10    14      821            825        -4      916            935
   arr_delay carrier flight tail

The one plane that is represented is N539AA. You can [look it up](https://flightaware.com/resources/registration/N539AA) in an online database and find out it's a private jet. My guess is the others are private jets too.

Picture: https://s3.eu-west-2.amazonaws.com/abpic-media-eu-production/pictures/full_size_0415/1625779-large.jpg


# SQL - Structured Query Language

A huge amount of data lives in relational databases so it is important to understand how to connect to a relational database and work with it.

To connect to the database from R, you’ll use a pair of packages:

* DBI (database interface) - this provides a set of generic functions that connect to the database, upload data, run SQL queries, etc.
* You’ll also use a package tailored for the DBMS you’re connecting to. This package translates the generic DBI commands into the specifics needed for a given DBMS. There’s usually one package for each DMBS, e.g. RPostgres for Postgres and RMariaDB for MySQL. In this example we use SQLite and the package to use is **RSQLite**


In [15]:
# Create an ephemeral in-memory RSQLite database
con <- dbConnect(RSQLite::SQLite(), ":memory:")

dbListTables(con)

character(0)

At the outset you see that there are no tables when you run the above command.

### dbplyr
dbplyr is a dplyr backend, that allows you to keep writing dplyr code and **dbplyr** translates it to SQL.
Now we are going to create a table called 'mpg' in our SQLite database using the mpg tibble and dbplyr is generating the necessary SQL queries to get this done behind the scenes.

In [16]:
dbWriteTable(con, "mpg", mpg, overwrite = T)
dbListTables(con)

[1] "mpg"

In [ ]:
# dbRemoveTable(con, 'mpg') # this will delete the table

Now let us look into the column names of this table

In [ ]:
dbListFields(con, "mpg")

[1] "manufacturer" "model"        "displ"        "year"         "cyl"         
 [6] "trans"        "drv"          "cty"          "hwy"          "fl"          
[11] "class"

Time to read the entire table!

In [17]:
dbReadTable(con, "mpg")

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact


### Selecting data from a table
The SQL syntax for selecting column(s) from a table is
```{sql}
SELECT <col1>, <col2>, ..., <coln> FROM <table>
```
Note the similarity to the corresponding `tidyverse` command:
```{r}
select(<table>, <col1>, <col2>, ..., <coln>)
```

Let us select some records

In [18]:
res <- dbSendQuery(con, "SELECT * FROM mpg")
res

<SQLiteResult>
  SQL  SELECT * FROM mpg
  ROWS Fetched: 0 [incomplete]
       Changed: 0

In [19]:
dbFetch(res) %>% head

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact


In [20]:
dbFetch(res)

manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>


The special keyword `*` means "select everything" and is equivalent to `dplyr`'s `everything()`:

If you have a really big table, SQL allows you to `LIMIT` the number of rows it returns.f

In [21]:
# using limit
res <- dbSendQuery(con, "SELECT * FROM mpg LIMIT 2")
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact


In [ ]:
dbClearResult(res)

### Filtering

The SQL syntax for filtering rows in a table uses the `WHERE` clause:
```{sql}
SELECT * FROM <table> WHERE dest = "IAH"
```
This is the same as:
```{r}
filter(<table>, dest == "IAH")
```
Note that SQL uses a single `=` to check equality!

In [22]:
res <- dbSendQuery(con, "SELECT * FROM mpg WHERE cyl = 4")
dbFetch(res)

Warning message:
“Closing open result set, pending rows”


manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact
audi,a4 quattro,2.0,2008,4,auto(s6),4,19,27,p,compact
chevrolet,malibu,2.4,1999,4,auto(l4),f,19,27,r,midsize


Now let us create more tables using our tidyverse datasets

In [23]:

dbWriteTable(con, "flights", flights, overwrite=T)
dbWriteTable(con, "airports", airports, overwrite=T)
dbWriteTable(con, "airlines", airlines, overwrite=T)
dbWriteTable(con, "planes", planes, overwrite=T)
dbWriteTable(con, "weather", weather, overwrite=T)

Warning message:
“Closing open result set, pending rows”


We will define a convenience function to simplify our query

In [24]:
q <- function(...) dbGetQuery(con, ...)
print(q)

function(...) dbGetQuery(con, ...)


### Missing data
In SQL, missing data is coded as `NULL`. This is a special value which is analogous to `NA` in R.

In [25]:
# select non-null dep time

q('SELECT * FROM flights WHERE dep_delay IS NOT NULL') %>% as_tibble %>% print

# filter(flights, !is.na(dep_delay))

# A tibble: 328,521 × 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# ℹ 328,511 more rows
# ℹ 1

### Summarizing

The SQL syntax for summarizing is using the `GROUP BY` clause:
```{sql}
SELECT AVG(<col>) AS avg_col FROM <table> GROUP BY(<group cols>)
```
This is the same as:
```{r}
<table> %>% group_by(<group cols>) %>% summarize(avg_col = mean(<col>))
```

In [28]:
# select the average distance and count for each day

q("SELECT year, month, day, AVG(distance) AS avg_dist,
    COUNT() AS n FROM flights GROUP BY year, month, day")

# equivalent to
# flights %>% group_by(year, month, day) %>% summarize(avg_dist = mean(distance), n = n())

year,month,day,avg_dist,n
<int>,<int>,<int>,<dbl>,<int>
2013,1,1,1077.4299,842
2013,1,2,1053.1177,943
2013,1,3,1037.3709,914
2013,1,4,1032.4754,915
2013,1,5,1067.5917,720
2013,1,6,1051.6466,832
2013,1,7,998.2572,933
2013,1,8,985.5328,899
2013,1,9,981.4202,902


### Joins

The SQL syntax for joins:
```{sql}
SELECT * FROM <table> LEFT JOIN <other_table> ON <left_key_col> = <right_key_col>
```
This is the same as:
```{r}
left_join(<table>, <other_table>, by = c("<left_key_col>" = "<right_key_col>"))
```

In [29]:
q("SELECT flights.year, planes.tailnum AS flightyr, * FROM flights
    LEFT JOIN planes ON flights.tailnum = planes.tailnum")

year,flightyr,year.1,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,⋯,time_hour,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,⋯,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>
2013,N14228,2013,1,1,517,515,2,830,819,⋯,1357034400,N14228,1999,Fixed wing multi engine,BOEING,737-824,2,149,NA,Turbo-fan
2013,N24211,2013,1,1,533,529,4,850,830,⋯,1357034400,N24211,1998,Fixed wing multi engine,BOEING,737-824,2,149,NA,Turbo-fan
2013,N619AA,2013,1,1,542,540,2,923,850,⋯,1357034400,N619AA,1990,Fixed wing multi engine,BOEING,757-223,2,178,NA,Turbo-fan
2013,N804JB,2013,1,1,544,545,-1,1004,1022,⋯,1357034400,N804JB,2012,Fixed wing multi engine,AIRBUS,A320-232,2,200,NA,Turbo-fan
2013,N668DN,2013,1,1,554,600,-6,812,837,⋯,1357038000,N668DN,1991,Fixed wing multi engine,BOEING,757-232,2,178,NA,Turbo-fan
2013,N39463,2013,1,1,554,558,-4,740,728,⋯,1357034400,N39463,2012,Fixed wing multi engine,BOEING,737-924ER,2,191,NA,Turbo-fan
2013,N516JB,2013,1,1,555,600,-5,913,854,⋯,1357038000,N516JB,2000,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-232,2,200,NA,Turbo-fan
2013,N829AS,2013,1,1,557,600,-3,709,723,⋯,1357038000,N829AS,1998,Fixed wing multi engine,CANADAIR,CL-600-2B19,2,55,NA,Turbo-fan
2013,N593JB,2013,1,1,557,600,-3,838,846,⋯,1357038000,N593JB,2004,Fixed wing multi engine,AIRBUS,A320-232,2,200,NA,Turbo-fan


### Ranking

Ranking operates slightly differently in SQL than dplyr. The overall syntax is:


    SELECT RANK() OVER (
	    PARTITION BY <columns>
	    ORDER BY <expression1> [ASC|DESC], ...
    ) RankedCol FROM <table>
    
    
The easiest way to learn it is to see some examples.

In [ ]:
# rank departure delay by month, day

q('SELECT year, month, day, dep_delay, RANK() OVER (
    PARTITION BY month, day
    ORDER BY dep_delay DESC
    ) r FROM flights')


year,month,day,dep_delay,r
<int>,<int>,<int>,<dbl>,<int>
2013,1,1,853,1
2013,1,1,379,2
2013,1,1,290,3
2013,1,1,285,4
2013,1,1,260,5
2013,1,1,255,6
2013,1,1,216,7
2013,1,1,192,8
2013,1,1,157,9


In [30]:
# The above query is equivalent to
flights %>% group_by(month, day) %>% mutate(r = min_rank(-dep_delay)) %>%
    select(month, day, dep_delay, r) %>% arrange(month, day, r)

month,day,dep_delay,r
<int>,<int>,<dbl>,<int>
1,1,853,1
1,1,379,2
1,1,290,3
1,1,285,4
1,1,260,5
1,1,255,6
1,1,216,7
1,1,192,8
1,1,157,9


## Subqueries

An aspect of SQL I find confusing is that you cannot filter (`WHERE ...`) based on a ranking condition:

In [ ]:
q('SELECT year, month, day, dep_delay, RANK() OVER (
    PARTITION BY month, day
    ORDER BY dep_delay DESC
    ) r FROM flights where r  = 1')

ERROR: Error: misuse of aliased window function r


You need to use subquery to get the result as shown below

In [31]:
q('SELECT year, month, day, dep_delay, r FROM
    (
        SELECT year, month, day, dep_delay, RANK() OVER (
        PARTITION BY month, day
        ORDER BY dep_delay DESC
    ) r FROM flights)
    WHERE r <= 2
')


year,month,day,dep_delay,r
<int>,<int>,<int>,<dbl>,<int>
2013,1,1,853,1
2013,1,1,379,2
2013,1,2,379,1
2013,1,2,337,2
2013,1,3,291,1
2013,1,3,268,2
2013,1,4,288,1
2013,1,4,208,2
2013,1,5,327,1


### Solve
Find the max highway miles across each manufacturer

In [32]:
res <- dbSendQuery(con, "select manufacturer, max(hwy) from mpg group by manufacturer")
dbFetch(res)
dbClearResult(res)

manufacturer,max(hwy)
<chr>,<int>
audi,31
chevrolet,30
dodge,24
ford,26
honda,36
hyundai,31
jeep,22
land rover,18
lincoln,18


## 🤔 Quiz

How many 4 cylinder models are present in this database?

<ol style="list-style-type: upper-alpha;">
    <li>81</li>
    <li>52</li>
    <li>63</li>
    <li>91</li>
</ol>

In [33]:
# fill in the blanks
res <- dbSendQuery(con, "select count(*) from mpg where cyl = 4")
dbFetch(res)
dbClearResult(res)

count(*)
<int>
81


### Solve
Extend the above query to retrieve the minimum hwy value across manufacturer and model

In [38]:
res <- dbSendQuery(con, "SELECT manufacturer, model, min(hwy) FROM mpg GROUP BY manufacturer, model")
dbFetch(res)
dbClearResult(res)

manufacturer,model,min(hwy)
<chr>,<chr>,<int>
audi,a4,26
audi,a4 quattro,25
audi,a6 quattro,23
chevrolet,c1500 suburban 2wd,15
chevrolet,corvette,23
chevrolet,k1500 tahoe 4wd,14
chevrolet,malibu,26
dodge,caravan 2wd,17
dodge,dakota pickup 4wd,12


### Solve
Get all the manufacturers who have 4 or more 'compact' car class

In [40]:
res <- dbSendQuery(con, "SELECT manufacturer, class, count(*) FROM mpg WHERE class = 'compact' GROUP BY manufacturer HAVING count(*) >= 4")
dbFetch(res)
dbClearResult(res)

manufacturer,class,count(*)
<chr>,<chr>,<int>
audi,compact,15
subaru,compact,4
toyota,compact,12
volkswagen,compact,14


### Solve
Get distinct car manufacturers in this dataset

In [41]:
res <- dbSendQuery(con, "SELECT DISTINCT manufacturer FROM mpg")
dbFetch(res)
dbClearResult(res)

manufacturer
<chr>
audi
chevrolet
dodge
ford
honda
hyundai
jeep
land rover
lincoln


Once done, it is a good idea to disconnect from the database

In [ ]:
dbClearResult(res)
dbDisconnect(con)